In [2]:
import numpy as np
import pandas as pd

In [3]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn import neighbors, model_selection, tree
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

In [13]:
dataset = pd.read_csv('all_stocks_5yr.csv', delimiter=',')
#se eliminan las filas que tienen valores nulos de cara a realizar posteriores estudios
#HABRIA QUE TENERLO IMPUTADO CON VALORES PERDIDOS!!
dataset.dropna(axis=0, inplace = True) 

In [14]:
atSalida = 'close'
atEntrada = ['open','high','low','volume']

dataset_output = dataset[atSalida].copy()
dataset.drop(atSalida, axis=1, inplace=True)
dataset = dataset[atEntrada].copy()

Por tanto , las columnas nulas son open, high y  low

# 3.3 Transformación de variables categóricas a numéricas

No se va a realizar dicho tipo de transformación , pues para el proyecto se realizará principalmente redes neuronales y regresión polinomial.

# 3.4 Reducción de datos

AGREGAR DESPUES VARIABLE FECHA!!!!!

# Obtención del rendimiento con todas las variables

Para comprobar la calidad de las técnicas de selección de variables, se va a calcular el rendimiento de una regresión polinomial  si utilizamos todas las variables. Nos hemos decidido por esta técnica, pues es fácil de implementar.

Se aplicará el método hold-out para obtener los conjuntos de entrenamiento y de test (70% de datos para entrenar).



SE ELIMINARAN LAS FILAS ASOCIADAS A LOS VALORES NULOS PARA PODER REALIZAR LAS COSAS!!! JUNTAR CON LO DE IVAN!

Rendimiento con regresión polinomial sobre todas las variables

In [16]:
# Generamos los conjuntos de entrenamiento y de test
X_train, X_test, y_train, y_test = train_test_split(dataset, dataset_output, test_size=0.2, random_state=12)

In [17]:
def regresion_polinomial():
    # Creamos una Pipeline en la que generamos variables polinómicas de grado 2, estandarizamos los datos y aprendemos una regresión lineal
    pipePolinomial = Pipeline([('polynomial', PolynomialFeatures(degree=2)),('scaler', StandardScaler()),('regressor', LinearRegression())])
    # Entrenamos la Pipeline
    pipePolinomial.fit(X_train, y_train)
    # Obtenemos el rendimiento en entrenamiento y en test (MSE)
    prTrain = pipePolinomial.predict(X_train)
    errorTrain = mean_squared_error(y_train, prTrain)
    prTest = pipePolinomial.predict(X_test)
    errorTest = mean_squared_error(y_test, prTest)
    print('El error en entrenamiento es {:.4f} y en test es {:.4f}'.format(errorTrain, errorTest))
regresion_polinomial()

El error en entrenamiento es 0.4663 y en test es 0.4903


## Análisis de correlaciones

In [ ]:
import matplotlib.pyplot as plt
correlaciones = X_train.corr(method ='pearson')
correlaciones

In [ ]:
# Mostramos la matriz de correlaciones especificando el rango de los valores [-1, 1]
fig = plt.figure(figsize=(50,50))
ax = fig.add_subplot(111)
cax = ax.matshow(correlaciones, vmin=-1, vmax=1, cmap=plt.cm.rainbow)
fig.colorbar(cax)
ticks = np.arange(0,len(X_train.columns),1)
ax.set_xticks(ticks)
ax.set_yticks(ticks)
# Añadimos los nombres de las variables en la figura
names = X_train.columns
ax.set_xticklabels(names, rotation='90')
ax.set_yticklabels(names)
plt.show()

Tras la obtención de matriz de correlaciones, se puede determinar que las variables de entrada están relacionadas entre si de manera muy alta, salvo el volumen.

Veamos ahora el rendimiento que se obtendría al eliminar una de éstas variables. Para hacer más completo el experimento, se procederá a crear 3 datasets distintos: cada uno eliminando una variable distinta.

*Eliminación* *de* *open*, *high*, *low* por separado

In [ ]:
dataset1 = dataset.copy()
dataset_output1 = dataset_output.copy()

In [ ]:
variables = ['open', 'high', 'low']   
for nombrevar in variables:
    X_train, X_test, y_train, y_test = train_test_split(dataset1, dataset_output1, test_size=0.2, random_state=12)
    X_train = X_train.drop([nombrevar],axis=1)
    X_test = X_test.drop([nombrevar],axis=1)

    pipePolinomial = Pipeline([('polynomial', PolynomialFeatures(degree=2)),('scaler', StandardScaler()),('regressor', LinearRegression())])
    pipePolinomial.fit(X_train, y_train)

    #Se obtiene el rendimiento en entrenamiento y en test (MSE)
    prTrain = pipePolinomial.predict(X_train)
    errorTrain = mean_squared_error(y_train, prTrain)
    prTest = pipePolinomial.predict(X_test)
    errorTest = mean_squared_error(y_test, prTest)
    print(f"Resutados para la eliminacion de {nombrevar}")
    print('El error en entrenamiento es {:.4f} y en test es {:.4f}'.format(errorTrain, errorTest))
    print()

*Eliminación* *por* *pares*

In [ ]:
pares = [ ['open', 'high'] , ['open', 'low'] , ['high', 'low']]   
for par in pares:
    X_train, X_test, y_train, y_test = train_test_split(dataset1, dataset_output1, test_size=0.2, random_state=12)
    
    X_train = X_train.drop([par[0]],axis=1)
    X_train = X_train.drop([par[1]],axis=1)

    X_test = X_test.drop([par[0]],axis=1)
    X_test = X_test.drop([par[1]],axis=1)


    pipePolinomial = Pipeline([('polynomial', PolynomialFeatures(degree=2)),('scaler', StandardScaler()),('regressor', LinearRegression())])
    pipePolinomial.fit(X_train, y_train)

    #Se obtiene el rendimiento en entrenamiento y en test (MSE)
    prTrain = pipePolinomial.predict(X_train)
    errorTrain = mean_squared_error(y_train, prTrain)
    prTest = pipePolinomial.predict(X_test)
    errorTest = mean_squared_error(y_test, prTest)
    print(f"Resutados para la eliminacion de {par[0]} y {par[1]}")
    print('El error en entrenamiento es {:.4f} y en test es {:.4f}'.format(errorTrain, errorTest))
    print()

*Eliminación* *de* *las* *tres* *variables*

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dataset1, dataset_output1, test_size=0.2, random_state=12)
    
X_train = X_train.drop(['open'] ,axis=1)
X_train = X_train.drop(['low'],axis=1)
X_train = X_train.drop(['high'] ,axis=1)

X_test = X_test.drop(['open'] ,axis=1)
X_test = X_test.drop(['low'],axis=1)
X_test = X_test.drop(['high'] ,axis=1)

pipePolinomial = Pipeline([('polynomial', PolynomialFeatures(degree=2)),('scaler', StandardScaler()),('regressor', LinearRegression())])
pipePolinomial.fit(X_train, y_train)

#Se obtiene el rendimiento en entrenamiento y en test (MSE)
prTrain = pipePolinomial.predict(X_train)
errorTrain = mean_squared_error(y_train, prTrain)
prTest = pipePolinomial.predict(X_test)
errorTest = mean_squared_error(y_test, prTest)
print("Resultado para la eliminacion de high, low, y open")
print('El error en entrenamiento es {:.4f} y en test es {:.4f}'.format(errorTrain, errorTest))
print()

Tras los experimentos realizados, se puede observar que se obtiene un peor resultado al tratar de eliminar cualquiera de las tres variables que cuentan con una mayor correlación -casi perfecta- .

Por otro lado, en caso de eliminar una variable,la que menos impacto tiene en el rendimiento del modelo es 'open', y la que más influye es 'high'. 

En caso de eliminar dos variables,el mayor impacto negativo en el rendimiento vendrá dado por la eliminación de conjunta de 'low' y 'high'. Cosa que es natural, pues representa el valor más bajo y más alto de las acciones de una empresa en un día.

La eliminación de 'high', 'low' y 'open' provoca un desenlace nefasto y esperado, pues predecir el valor al final del día de las acciones a partir del volumen de acciones negociadas es sumamente complicado 

# Comparativas de métodos

# Uso de regresión polinomial

In [18]:
regresion_polinomial()

El error en entrenamiento es 0.4663 y en test es 0.4903


# LSTM

In [22]:
#importing the packages 
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM

In [ ]:
pip install keras
pip install tensorflow

In [29]:
#Lectura de datos
todataframe = pd.read_csv('all_stocks_5yr.csv', delimiter=',')
#se eliminan las filas que tienen valores nulos de cara a realizar posteriores estudios
#HABRIA QUE TENERLO IMPUTADO CON VALORES PERDIDOS!!
todataframe.dropna(axis=0, inplace = True) 
#atSalida = 'close'
#atEntrada = ['date','open','high','low','volume']
#dataset_output = dataset[atSalida].copy()

La comparativa del método se hará con la empresa AAl

In [30]:
todataframe = todataframe[todataframe['Name']=='AAL']

In [31]:
#Creacion del dataframe
seriesdata = todataframe.sort_index(ascending=True, axis=0)
new_seriesdata = pd.DataFrame(index=range(0,len(todataframe)),columns=['date','close'])
length_of_data=len(seriesdata)
for i in range(0,length_of_data):
    new_seriesdata['date'][i] = seriesdata['date'][i]
    new_seriesdata['close'][i] = seriesdata['close'][i]
new_seriesdata

,date,close
0,2013-02-08,14.75
1,2013-02-11,14.46
2,2013-02-12,14.27
3,2013-02-13,14.66
4,2013-02-14,13.99
...,...,...
1254,2018-02-01,53.88
1255,2018-02-02,52.1
1256,2018-02-05,49.76
1257,2018-02-06,51.18


In [32]:
#Colocar los indices de nuevo
new_seriesdata.index = new_seriesdata.date
new_seriesdata.drop('date', axis=1, inplace=True)
#Crear un conjunto de entrenamiento y de test (70% para train, 30% para test)
numero_ejemplos_entrenamiento = int(len(new_seriesdata)*0.7)

In [35]:
#creating train and test sets this comprises the entire data’s present in the dataset
myseriesdataset = new_seriesdata.values
totrain = myseriesdataset[0:numero_ejemplos_entrenamiento,:]
tovalid = myseriesdataset[numero_ejemplos_entrenamiento:,:]

#converting dataset into x_train and y_train
scalerdata = MinMaxScaler(feature_range=(0, 1))
scale_data = scalerdata.fit_transform(myseriesdataset)
x_totrain, y_totrain = [], []
length_of_totrain=len(totrain)
for i in range(60,length_of_totrain):
    x_totrain.append(scale_data[i-60:i,0])
    y_totrain.append(scale_data[i,0])
x_totrain, y_totrain = np.array(x_totrain), np.array(y_totrain)
x_totrain = np.reshape(x_totrain, (x_totrain.shape[0],x_totrain.shape[1],1))

In [40]:
#LSTM neural network
lstm_model = Sequential()
lstm_model.add(LSTM(units=50, return_sequences=True, input_shape=(x_totrain.shape[1],1)))
lstm_model.add(LSTM(units=50))
lstm_model.add(Dense(1))
lstm_model.compile(loss='mean_squared_error', optimizer='adadelta')
lstm_model.fit(x_totrain, y_totrain, epochs=3, batch_size=1, verbose=2)

Epoch 1/3
821/821 - 25s - loss: 0.1847
Epoch 2/3
821/821 - 23s - loss: 0.1254
Epoch 3/3
821/821 - 23s - loss: 0.0740


In [41]:
#Predecir
myinputs = new_seriesdata[len(new_seriesdata) - (len(tovalid)+1) - 60:].values
myinputs = myinputs.reshape(-1,1)
myinputs  = scalerdata.transform(myinputs)
tostore_test_result = []
for i in range(60,myinputs.shape[0]):
    tostore_test_result.append(myinputs[i-60:i,0])

In [46]:
tostore_test_result = np.array(tostore_test_result)
tostore_test_result = np.reshape(tostore_test_result,(tostore_test_result.shape[0],tostore_test_result.shape[1],1))

In [49]:
myclosing_priceresult = lstm_model.predict(tostore_test_result)
myclosing_priceresult

array([[0.3102483 ],
       [0.31000102],
       [0.30972663],
       [0.30935878],
       [0.30911124],
       [0.3089827 ],
       [0.30917886],
       [0.3098604 ],
       [0.31088534],
       [0.31214866],
       [0.3135089 ],
       [0.31477386],
       [0.31600252],
       [0.3172405 ],
       [0.31847778],
       [0.31956786],
       [0.3204845 ],
       [0.32145154],
       [0.32228884],
       [0.32296646],
       [0.3235718 ],
       [0.3242212 ],
       [0.3253274 ],
       [0.32696044],
       [0.32877177],
       [0.330594  ],
       [0.33213365],
       [0.33314425],
       [0.33368975],
       [0.33365527],
       [0.33314317],
       [0.3322216 ],
       [0.3309017 ],
       [0.3294472 ],
       [0.3280333 ],
       [0.32656154],
       [0.32532477],
       [0.324302  ],
       [0.32346058],
       [0.3229642 ],
       [0.32302636],
       [0.3235294 ],
       [0.32440025],
       [0.32559475],
       [0.32689968],
       [0.32849735],
       [0.33014894],
       [0.331

In [51]:
myclosing_priceresult = scalerdata.inverse_transform(myclosing_priceresult)

In [52]:
myclosing_priceresult.shape

(379, 1)